In [121]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from tqdm import tqdm
import datetime as dt
data = pd.read_csv('data/train.csv')

In [2]:
data.head()

,Title,Address.1st.Line,Address.2nd.Line,City,State,Zip.Code,License.Type,License.Issue.Date,Dental.School,Graduation.Date,broker.ID,approach.date,convert
0,DMD,6335 E. Mountain View,NaN,Paradise Valley,AZ,85253,Dentist License,1987-06-10,U of Pittsburg,01/01/1970,chris,2017-08-04,1
1,DDS,24 N Bullmoose Cir,NaN,Chandler,AZ,85224,Dentist License,2009-06-11,SUNY,06/01/2007,chris,2017-06-17,0
2,DDS,2470 W Ray Road Suite 1,NaN,Chandler,AZ,85224,Dentist License,2001-07-16,U of TX/SA,05/26/2001,karl,2017-07-03,0
3,DDS,6649 East Valle Di Cadore,NaN,Tucson,AZ,85750,Dentist License,1963-01-01,NW,01/01/1957,chris,2016-11-14,0
4,DDS,6938 E. Gary,NaN,Scottsdale,AZ,85254,Dentist License,1979-12-13,U of Detroit,01/01/1965,sarah,2017-08-27,0


In [3]:
def date_to_nth_day(date, format='%Y-%m-%d'):
    date = pd.to_datetime(date, format=format)
    new_year_day = pd.Timestamp(year=date.year, month=1, day=1)
    return int((date - new_year_day).days + 1)

# Train

In [4]:
data = data[data['convert']==1]

data.drop(labels=['License.Type','State', 'Address.1st.Line','Address.2nd.Line','Zip.Code'],axis=1, inplace=True)

#Handling Approach
data['approach']=data['approach.date'].apply(lambda x:date_to_nth_day(x))
data.drop(labels=['approach.date'],axis=1, inplace=True)

#Handling License and Graudation date
data['licensedate']=data['License.Issue.Date'].apply(lambda x: x[:4])
data['licensedate'] = data['licensedate'].astype(int)
data['graduation']=data['Graduation.Date'].apply(lambda x: x[-4:])
data['graduation'] = data['graduation'].astype(int)
data.drop(labels=['License.Issue.Date'],axis=1, inplace=True)
data.drop(labels=['Graduation.Date'],axis=1, inplace=True)

correct_index = ['Title', 'City', 'Dental.School', 'licensedate', 'graduation']
data = data[correct_index]
data = pd.concat([data,pd.get_dummies(data['Title']),pd.get_dummies(data['City']),pd.get_dummies(data['Dental.School']),
                        pd.get_dummies(data['licensedate']),
    pd.get_dummies(data['graduation'])],axis=1)
data.reset_index(drop=True, inplace=True)
train_set=set(list(data.columns.values))

# With Broker ID

In [124]:
# trainwbroker = data[data['convert']==1]
trainwbroker['approach']=trainwbroker['approach.date'].apply(lambda x:date_to_nth_day(x))
# trainwbroker.reset_index(inplace=True, drop=True)

/home/jayrodge/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [125]:
trainwbroker.head()

,Title,Address.1st.Line,Address.2nd.Line,City,State,Zip.Code,License.Type,License.Issue.Date,Dental.School,Graduation.Date,broker.ID,approach.date,convert,approach
0,DMD,6335 E. Mountain View,NaN,Paradise Valley,AZ,85253,Dentist License,1987-06-10,U of Pittsburg,01/01/1970,chris,2017-08-04,1,216
1,DMD,5620 W Thunderbird Rd Ste G-1,NaN,Glendale,AZ,85306,Dentist License,2006-04-10,U of PA,05/13/2002,karl,2017-01-31,1,31
2,DDS,3202 E. Greenway Rd,Ste 1287,Phoenix,AZ,85032,Dentist License,1988-11-18,Loyola/Chicago,05/31/1987,karl,2018-12-25,1,359
3,DDS,2474 E Calle Los Altos,NaN,Tucson,AZ,85718,Dentist License,2005-05-13,U of TX/H,05/24/2003,chris,2016-01-17,1,17
4,DDS,3100 W. Renegade Rd.,NaN,Prescott,AZ,86305,Dentist License,1993-02-25,Creighton,05/08/1991,karl,2016-06-26,1,178


# Test

In [5]:
test = pd.read_csv('data/test.csv')
test.drop(labels=['License.Type','State', 'Address.1st.Line','Address.2nd.Line','Zip.Code'],axis=1, inplace=True)
#Handling License and Graudation date
test['licensedate']=test['License.Issue.Date'].apply(lambda x: x[:4])
test['licensedate'] = test['licensedate'].astype(int)
test['graduation']=test['Graduation.Date'].apply(lambda x: x[-4:])
test['graduation'] = test['graduation'].astype(int)
test.drop(labels=['License.Issue.Date'],axis=1, inplace=True)
test.drop(labels=['Graduation.Date'],axis=1, inplace=True)

correct_index = ['Title', 'City', 'Dental.School', 'licensedate', 'graduation']
test = test[correct_index]
test = pd.concat([test,pd.get_dummies(test['Title']),pd.get_dummies(test['City']),pd.get_dummies(test['Dental.School']),
                        pd.get_dummies(test['licensedate']),    pd.get_dummies(test['graduation'])],axis=1)
test_set = set(list(test.columns.values))

In [6]:
total_columns = test_set | train_set

In [7]:
train_df = pd.DataFrame(0,columns=total_columns,index=np.arange(238))
test_df = pd.DataFrame(0,columns=total_columns,index=np.arange(len(test)))

In [8]:
data.head()

,Title,City,Dental.School,licensedate,graduation,BDS,DDS,DMD,Ajo,Casa Grande,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,DMD,Paradise Valley,U of Pittsburg,1987,1970,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,DMD,Glendale,U of PA,2006,2002,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,DDS,Phoenix,Loyola/Chicago,1988,1987,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,DDS,Tucson,U of TX/H,2005,2003,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,DDS,Prescott,Creighton,1993,1991,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
for i in tqdm(range(0,len(data))):
    train_df.loc[i][data.loc[i]['Title']] = 1
    train_df.loc[i][data.loc[i]['City']] = 1
    train_df.loc[i][data.loc[i]['Dental.School']] = 1
    train_df.loc[i][data.loc[i]['licensedate']] = 1
    train_df.loc[i][data.loc[i]['graduation']] = 1

100%|██████████| 238/238 [00:03<00:00, 85.71it/s]


In [10]:
for i in tqdm(range(0,len(test))):
    test_df.loc[i][test.loc[i]['Title']] = 1
    test_df.loc[i][test.loc[i]['City']] = 1
    test_df.loc[i][test.loc[i]['Dental.School']] = 1
    test_df.loc[i][test.loc[i]['licensedate']] = 1
    test_df.loc[i][test.loc[i]['graduation']] = 1

100%|██████████| 2331/2331 [07:32<00:00,  5.17it/s]


In [105]:
test_df.drop(labels=['broker.ID','approach'],axis=1,inplace=True)

In [128]:
A = {}
B = {}
hawadf = {}
hawaapp = {}
CWBroker = Counter()
for test_index, test_row in tqdm(test_df.iterrows()):
    brokerid = []
    sum1 = 0
    average1 = 0
    approachdate = []
    for train_index, train_row in train_df.iterrows():
        A[train_index] = cosine_similarity([test_row], [train_row])
    
    B = dict(Counter(A).most_common(5))
    chris = 0
    karl = 0
    sarah = 0
    for key, value in B.items():
        stra = trainwbroker.loc[key]['broker.ID']
        if stra == "chris":
            chris= chris+1
        elif stra == "karl":
            karl= karl+1
        elif stra == "sarah":
            sarah= sarah+1
            
        sum1 = sum1 + trainwbroker.loc[key]['approach']
    average1 = int(sum1/5.0)
    
    
    if(chris > karl & chris > sarah):
        hawadf.update({test_index: "chris"})
        hawaapp.update({test_index: (dt.datetime(2019,2,1) + dt.timedelta(average1-1)).strftime('%Y-%m-%d')})
    elif(karl > sarah & karl > chris):
        hawadf.update({test_index: "karl"})
        hawaapp.update({test_index: (dt.datetime(2019,2,1) + dt.timedelta(average1-1)).strftime('%Y-%m-%d')})
    elif(sarah > karl & sarah > chris):
        hawadf.update({test_index: "sarah"})
        hawaapp.update({test_index: (dt.datetime(2019,2,1) + dt.timedelta(average1-1)).strftime('%Y-%m-%d')})
        
   

2331it [05:13,  7.44it/s]


In [130]:
len(hawaapp)

355

In [131]:
test_hawa_df = pd.read_csv('data/test.csv')

In [154]:
for key, value in hawadf.items():
    #print(test_hawa_df.loc[key]['broker.ID'])
    test_hawa_df.loc[key]['broker.ID'] = value
for key, value in tqdm(hawaapp.items()):
    test_hawa_df.loc[key]['approach.date'] = value

100%|██████████| 355/355 [00:00<00:00, 6641.92it/s]


In [153]:
test_hawa_df['broker.ID'] = 'NA'
test_hawa_df['approach.date'] = '0000-00-00'

In [161]:
test_hawa_df=test_hawa_df[test_hawa_df['broker.ID']!='NA']

In [162]:
test_hawa_df.to_csv('leads_rec.csv',index=False, index_label=False)